# Cloud Advanced Downloader v2.0
### Professional Multi-Engine Download Manager for Google Colab

---
**Features:** Multi-Engine | Resume Support | Mega Bypass | Real-time Progress | Drive Integration

**Repository:** [github.com/yourusername/Cloud-Advanced-Downloader](https://github.com)

## Cell 1: Mount Google Drive (Optional)

Toggle switch to enable/disable Google Drive mounting. Files are saved to `/content/download` by default.

In [ ]:
# =============================================================================
# CELL 1: MOUNT GOOGLE DRIVE (OPTIONAL)
# =============================================================================

from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
import os
import time

# Custom CSS for modern styling
custom_css = '''
<style>
    .cell-header {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 20px;
        border-radius: 12px;
        margin-bottom: 20px;
        font-family: 'Segoe UI', sans-serif;
        box-shadow: 0 4px 15px rgba(0,0,0,0.2);
    }
    .cell-header h2 { margin: 0; font-weight: 300; letter-spacing: 1px; }
    .info-box {
        background: #f8f9fa;
        border-left: 4px solid #667eea;
        padding: 15px;
        margin: 15px 0;
        border-radius: 0 8px 8px 0;
    }
    .success-box {
        background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%);
        color: white;
        padding: 20px;
        border-radius: 12px;
        text-align: center;
        animation: fadeIn 0.5s ease;
    }
    @keyframes fadeIn { from { opacity: 0; transform: translateY(-10px); } to { opacity: 1; transform: translateY(0); } }
    .loading-container {
        display: flex;
        align-items: center;
        justify-content: center;
        padding: 30px;
    }
    .spinner {
        border: 4px solid #f3f3f3;
        border-top: 4px solid #667eea;
        border-radius: 50%;
        width: 50px;
        height: 50px;
        animation: spin 1s linear infinite;
        margin-right: 20px;
    }
    @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } }
</style>
'''

display(HTML(custom_css))

# Header
display(HTML('''
<div class="cell-header">
    <h2>Step 1: Google Drive Integration</h2>
    <p style="margin: 10px 0 0 0; opacity: 0.9;">Mount your Google Drive to enable file transfer capabilities</p>
</div>
'''))

# Toggle widget for Drive mount
mount_toggle = widgets.ToggleButton(
    value=False,
    description='Mount Drive: OFF',
    button_style='warning',
    icon='unlink',
    layout=widgets.Layout(width='200px', height='45px')
)

# Status output
status_output = widgets.Output()

def on_toggle_change(change):
    if change['new']:  # ON state
        mount_toggle.description = 'Mount Drive: ON'
        mount_toggle.button_style = 'success'
        mount_toggle.icon = 'link'
        
        with status_output:
            clear_output()
            display(HTML('''
                <div class="loading-container">
                    <div class="spinner"></div>
                    <div>
                        <h3 style="margin: 0; color: #667eea;">Mounting Google Drive...</h3>
                        <p style="margin: 5px 0 0 0; color: #666;">Please authorize in the popup window</p>
                    </div>
                </div>
            '''))
        
        try:
            from google.colab import drive
            drive.mount('/content/drive', force_remount=True)
            time.sleep(1)
            
            with status_output:
                clear_output()
                display(HTML('''
                    <div class="success-box">
                        <h3 style="margin: 0 0 10px 0;">Drive Mounted Successfully!</h3>
                        <p style="margin: 0;">Path: <code>/content/drive/MyDrive</code></p>
                    </div>
                '''))
        except Exception as e:
            with status_output:
                clear_output()
                display(HTML(f'''
                    <div style="background: #ff6b6b; color: white; padding: 20px; border-radius: 12px;">
                        <h3 style="margin: 0 0 10px 0;">Mount Failed</h3>
                        <p style="margin: 0;">{str(e)}</p>
                    </div>
                '''))
                mount_toggle.value = False
    else:  # OFF state
        mount_toggle.description = 'Mount Drive: OFF'
        mount_toggle.button_style = 'warning'
        mount_toggle.icon = 'unlink'
        with status_output:
            clear_output()

mount_toggle.observe(on_toggle_change, names='value')

# Info box
info_html = '''
<div class="info-box">
    <strong>Default Download Location:</strong><br>
    <code>/content/download</code> (temporary, cleared on session end)<br><br>
    <strong>Google Drive Location:</strong><br>
    <code>/content/drive/MyDrive</code> (persistent storage)
</div>
'''

# Display widgets
display(widgets.VBox([
    widgets.HBox([mount_toggle], layout=widgets.Layout(justify_content='center', padding='20px')),
    status_output
]))
display(HTML(info_html))

# Create default download directory
os.makedirs('/content/download', exist_ok=True)
print("Default download directory ready: /content/download")

## Cell 2: Installation & Engine Setup

Install and configure all download engines including Aria2, wget, curl, and Mega tools.

In [ ]:
# =============================================================================
# CELL 2: INSTALLATION & SETUP ENGINE
# =============================================================================

from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
import subprocess
import os
import time
import threading

# Output container
setup_output = widgets.Output()
progress_output = widgets.Output()

# Custom styling
setup_css = '''
<style>
    .setup-header {
        background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%);
        color: white;
        padding: 20px;
        border-radius: 12px;
        margin-bottom: 20px;
        font-family: 'Segoe UI', sans-serif;
    }
    .step-card {
        background: white;
        border-radius: 10px;
        padding: 15px;
        margin: 10px 0;
        box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        border-left: 4px solid #667eea;
        transition: all 0.3s ease;
    }
    .step-card:hover { transform: translateX(5px); box-shadow: 0 4px 12px rgba(0,0,0,0.15); }
    .step-card.completed { border-left-color: #38ef7d; }
    .step-card.running { border-left-color: #ffe66d; animation: pulse 1.5s infinite; }
    .step-card.pending { border-left-color: #ccc; opacity: 0.7; }
    @keyframes pulse { 0%, 100% { opacity: 1; } 50% { opacity: 0.7; } }
    .status-icon { font-size: 1.5em; margin-right: 10px; }
    .console-output {
        background: #1e1e1e;
        color: #d4d4d4;
        padding: 15px;
        border-radius: 8px;
        font-family: 'Consolas', monospace;
        font-size: 0.85em;
        max-height: 200px;
        overflow-y: auto;
        margin-top: 10px;
    }
    .progress-ring {
        display: inline-block;
        width: 60px;
        height: 60px;
        margin: 20px auto;
    }
    .progress-ring circle {
        fill: none;
        stroke-width: 4;
        stroke-linecap: round;
        animation: rotate 2s linear infinite;
    }
    .progress-ring .background { stroke: #e0e0e0; }
    .progress-ring .progress { stroke: #667eea; stroke-dasharray: 150; stroke-dashoffset: 50; }
    @keyframes rotate { 100% { transform: rotate(360deg); } }
</style>
'''

display(HTML(setup_css))

# Header
display(HTML('''
<div class="setup-header">
    <h2 style="margin: 0;">Step 2: Engine Installation</h2>
    <p style="margin: 10px 0 0 0; opacity: 0.9;">Installing and configuring download engines</p>
</div>
'''))

# Installation steps
steps = [
    ('System Dependencies', 'apt-get update & install core packages'),
    ('Download Tools', 'aria2, wget, curl, pv, rsync'),
    ('Python Packages', 'requests, tqdm, beautifulsoup4'),
    ('Mega Tools', 'megatools for Mega.nz downloads'),
    ('Aria2 Configuration', 'RPC setup with optimal settings'),
    ('Directory Setup', 'Create /content/download folder')
]

step_widgets = []
for i, (title, desc) in enumerate(steps):
    step_html = widgets.HTML(f'''
        <div class="step-card pending" id="step-{i}">
            <div style="display: flex; align-items: center;">
                <span class="status-icon" id="icon-{i}">⏳</span>
                <div>
                    <strong style="font-size: 1.1em;">{title}</strong>
                    <p style="margin: 5px 0 0 0; color: #666; font-size: 0.9em;">{desc}</p>
                </div>
            </div>
        </div>
    ''')
    step_widgets.append(step_html)

steps_container = widgets.VBox(step_widgets)
display(steps_container)

# Progress indicator
with progress_output:
    display(HTML('''
        <div style="text-align: center; padding: 20px;">
            <svg class="progress-ring" viewBox="0 0 60 60">
                <circle class="background" cx="30" cy="30" r="25"></circle>
                <circle class="progress" cx="30" cy="30" r="25"></circle>
            </svg>
            <p style="color: #667eea; font-weight: bold;">Preparing installation...</p>
        </div>
    '''))

display(progress_output)
display(setup_output)

def update_step(index, status):
    icons = {'pending': '⏳', 'running': '⏳', 'completed': '✅', 'error': '❌'}
    colors = {'pending': '#ccc', 'running': '#ffe66d', 'completed': '#38ef7d', 'error': '#ff6b6b'}
    
    step_widget = step_widgets[index]
    step_widget.value = step_widget.value.replace('pending', status).replace('running', status if status != 'running' else 'running')
    step_widget.value = step_widget.value.replace(f'id="icon-{index}">⏳', f'id="icon-{index}">{icons[status]}')
    step_widget.value = step_widget.value.replace(f'id="icon-{index}">✅', f'id="icon-{index}">{icons[status]}')

def run_installation():
    logs = []
    
    def log(msg):
        logs.append(msg)
        with setup_output:
            clear_output(wait=True)
            display(HTML(f'<div class="console-output">{"<br>".join(logs[-15:])}</div>'))
    
    try:
        # Step 1: System dependencies
        update_step(0, 'running')
        log('[1/6] Updating package list...')
        subprocess.run(['apt-get', 'update', '-qq'], capture_output=True)
        log('[1/6] Package list updated ✓')
        update_step(0, 'completed')
        
        # Step 2: Download tools
        update_step(1, 'running')
        log('[2/6] Installing download tools...')
        subprocess.run(['apt-get', 'install', '-y', '-qq', 'aria2', 'wget', 'curl', 'pv', 'rsync', 'p7zip-full'],
                      capture_output=True)
        log('[2/6] Download tools installed ✓')
        update_step(1, 'completed')
        
        # Step 3: Python packages
        update_step(2, 'running')
        log('[3/6] Installing Python packages...')
        subprocess.run(['pip', 'install', '-q', 'requests', 'tqdm', 'beautifulsoup4', 'lxml', 'mega.py', 'urllib3'],
                      capture_output=True)
        log('[3/6] Python packages installed ✓')
        update_step(2, 'completed')
        
        # Step 4: Mega tools
        update_step(3, 'running')
        log('[4/6] Installing Mega tools...')
        result = subprocess.run(['apt-get', 'install', '-y', '-qq', 'megatools'], capture_output=True)
        if result.returncode != 0:
            log('[4/6] Building megatools from source...')
            subprocess.run(['apt-get', 'install', '-y', '-qq', 'build-essential', 'libglib2.0-dev', 'libssl-dev'],
                          capture_output=True)
        log('[4/6] Mega tools ready ✓')
        update_step(3, 'completed')
        
        # Step 5: Aria2 configuration
        update_step(4, 'running')
        log('[5/6] Configuring Aria2...')
        
        aria2_conf = '''
dir=/content/download
max-concurrent-downloads=5
max-connection-per-server=16
split=16
min-split-size=10M
continue=true
max-tries=10
retry-wait=5
timeout=60
enable-rpc=true
rpc-listen-port=6800
rpc-max-request-size=1024M
rpc-allow-origin-all=true
rpc-secret=cloud_downloader_2024
log-level=warn
disk-cache=64M
file-allocation=falloc
user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36
'''
        
        os.makedirs(os.path.expanduser('~/.aria2'), exist_ok=True)
        with open(os.path.expanduser('~/.aria2/aria2.conf'), 'w') as f:
            f.write(aria2_conf)
        
        # Start Aria2 daemon
        subprocess.Popen(['aria2c', '--conf-path=' + os.path.expanduser('~/.aria2/aria2.conf'), '--daemon'],
                        capture_output=True)
        time.sleep(2)
        log('[5/6] Aria2 configured and started ✓')
        update_step(4, 'completed')
        
        # Step 6: Directory setup
        update_step(5, 'running')
        log('[6/6] Creating download directory...')
        os.makedirs('/content/download', exist_ok=True)
        log('[6/6] Directory ready: /content/download ✓')
        update_step(5, 'completed')
        
        # Final summary
        with progress_output:
            clear_output()
            display(HTML('''
                <div style="background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%);
                            color: white; padding: 30px; border-radius: 15px; text-align: center;
                            margin-top: 20px; animation: fadeIn 0.5s ease;">
                    <h2 style="margin: 0 0 15px 0;">Installation Complete!</h2>
                    <p style="margin: 0; opacity: 0.9;">All engines are ready. Proceed to Cell 3 to start downloading.</p>
                </div>
            '''))
        
    except Exception as e:
        log(f'Error: {str(e)}')

# Run installation in background thread
thread = threading.Thread(target=run_installation)
thread.start()

## Cell 3: Advanced Download Manager

Main download interface with multi-engine support, real-time progress tracking, and batch processing.

In [ ]:
# =============================================================================
# CELL 3: ADVANCED DOWNLOAD MANAGER
# =============================================================================

from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
import subprocess
import os
import re
import time
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

# CSS Styling
download_css = '''
<style>
    .download-header {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 25px;
        border-radius: 15px;
        margin-bottom: 25px;
        text-align: center;
    }
    .engine-selector {
        background: white;
        border-radius: 12px;
        padding: 20px;
        margin: 15px 0;
        box-shadow: 0 4px 15px rgba(0,0,0,0.1);
    }
    .engine-card {
        background: linear-gradient(135deg, #f5f7fa 0%, #e4e8ec 100%);
        border-radius: 10px;
        padding: 15px;
        margin: 10px 0;
        border-left: 5px solid #667eea;
    }
    .engine-card.aria2 { border-left-color: #e74c3c; }
    .engine-card.wget { border-left-color: #3498db; }
    .engine-card.curl { border-left-color: #9b59b6; }
    .engine-card.mega { border-left-color: #d35400; }
    .input-section {
        background: white;
        border-radius: 12px;
        padding: 20px;
        margin: 15px 0;
        box-shadow: 0 4px 15px rgba(0,0,0,0.1);
    }
    .url-input {
        width: 100%;
        padding: 12px;
        border: 2px solid #e0e0e0;
        border-radius: 8px;
        font-family: 'Consolas', monospace;
        font-size: 0.9em;
        transition: border-color 0.3s;
    }
    .url-input:focus {
        outline: none;
        border-color: #667eea;
    }
    .progress-container {
        background: white;
        border-radius: 12px;
        padding: 20px;
        margin: 15px 0;
        box-shadow: 0 4px 15px rgba(0,0,0,0.1);
    }
    .progress-bar-custom {
        height: 30px;
        background: #e0e0e0;
        border-radius: 15px;
        overflow: hidden;
        position: relative;
    }
    .progress-fill {
        height: 100%;
        background: linear-gradient(90deg, #667eea 0%, #764ba2 100%);
        border-radius: 15px;
        transition: width 0.5s ease;
        display: flex;
        align-items: center;
        justify-content: center;
        color: white;
        font-weight: bold;
    }
    .download-item {
        background: #f8f9fa;
        border-radius: 10px;
        padding: 15px;
        margin: 10px 0;
        border-left: 4px solid #667eea;
    }
    .download-item.completed { border-left-color: #27ae60; }
    .download-item.failed { border-left-color: #e74c3c; }
    .download-item.downloading { border-left-color: #f39c12; }
    .stats-grid {
        display: grid;
        grid-template-columns: repeat(auto-fit, minmax(150px, 1fr));
        gap: 15px;
        margin: 20px 0;
    }
    .stat-card {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 15px;
        border-radius: 10px;
        text-align: center;
    }
    .stat-value {
        font-size: 1.8em;
        font-weight: bold;
    }
    .stat-label {
        font-size: 0.85em;
        opacity: 0.9;
        margin-top: 5px;
    }
    .btn-download {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border: none;
        padding: 15px 40px;
        border-radius: 30px;
        font-size: 1.1em;
        font-weight: bold;
        cursor: pointer;
        transition: transform 0.3s, box-shadow 0.3s;
    }
    .btn-download:hover {
        transform: translateY(-2px);
        box-shadow: 0 8px 25px rgba(102, 126, 234, 0.4);
    }
    .slider-container {
        background: #f8f9fa;
        padding: 15px;
        border-radius: 10px;
        margin: 10px 0;
    }
</style>
'''

display(HTML(download_css))

# Header
display(HTML('''
<div class="download-header">
    <h1 style="margin: 0 0 10px 0;">Download Manager</h1>
    <p style="margin: 0; opacity: 0.9;">Multi-engine download with real-time progress tracking</p>
</div>
'''))

# Engine Information Dictionary
ENGINE_INFO = {
    'aria2': {
        'name': 'Aria2',
        'color': '#e74c3c',
        'desc': 'Ultra-fast multi-threaded downloader (up to 32 connections)',
        'pros': ['Multi-threaded', 'Resume support', 'RPC control', 'Best for large files'],
        'cons': ['Requires configuration']
    },
    'wget': {
        'name': 'Wget',
        'color': '#3498db',
        'desc': 'Simple and reliable HTTP/HTTPS/FTP downloader',
        'pros': ['Simple', 'Stable', 'Widely supported'],
        'cons': ['Single connection']
    },
    'curl': {
        'name': 'cURL',
        'color': '#9b59b6',
        'desc': 'Powerful data transfer with multiple protocol support',
        'pros': ['Versatile', 'Protocol rich', 'Flexible'],
        'cons': ['Single connection', 'Complex syntax']
    },
    'mega': {
        'name': 'Mega Auto',
        'color': '#d35400',
        'desc': 'Specialized Mega.nz downloader with quota bypass',
        'pros': ['No quota limit', 'Resume support', 'Auto-extract'],
        'cons': ['Mega only', 'Slower than direct']
    }
}

# Widgets
engine_dropdown = widgets.Dropdown(
    options=[
        ('⚡ Aria2 (Recommended)', 'aria2'),
        ('🌐 Wget (Simple)', 'wget'),
        ('📡 cURL (Flexible)', 'curl'),
        ('📦 Mega Auto (Mega.nz)', 'mega')
    ],
    value='aria2',
    description='Engine:',
    layout=widgets.Layout(width='300px')
)

mode_toggle = widgets.ToggleButtons(
    options=[('Serentak', 'parallel'), ('Bergilir', 'sequential')],
    value='parallel',
    description='Mode:',
    layout=widgets.Layout(margin='10px 0')
)

# Aria2 configuration sliders
connections_slider = widgets.IntSlider(
    value=16,
    min=1,
    max=32,
    step=1,
    description='Koneksi:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

split_slider = widgets.IntSlider(
    value=16,
    min=1,
    max=32,
    step=1,
    description='Split:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

# URL Inputs (up to 5)
url_inputs = []
for i in range(5):
    url_input = widgets.Text(
        placeholder=f'Link download {i+1} (opsional)',
        layout=widgets.Layout(width='100%'),
        style={'description_width': '80px'}
    )
    url_inputs.append(url_input)

# Output containers
engine_info_output = widgets.Output()
download_output = widgets.Output()
progress_output = widgets.Output()

def update_engine_info(change):
    engine = change['new']
    info = ENGINE_INFO[engine]
    
    with engine_info_output:
        clear_output(wait=True)
        display(HTML(f'''
            <div class="engine-card {engine}">
                <h4 style="margin: 0 0 10px 0; color: {info['color']};">{info['name']}</h4>
                <p style="margin: 0 0 10px 0; color: #666;">{info['desc']}</p>
                <div style="display: flex; gap: 20px; font-size: 0.85em;">
                    <div>
                        <strong style="color: #27ae60;">Pros:</strong>
                        <ul style="margin: 5px 0; padding-left: 18px;">
                            {''.join(f'<li>{pro}</li>' for pro in info['pros'])}
                        </ul>
                    </div>
                    <div>
                        <strong style="color: #e74c3c;">Cons:</strong>
                        <ul style="margin: 5px 0; padding-left: 18px;">
                            {''.join(f'<li>{con}</li>' for con in info['cons'])}
                        </ul>
                    </div>
                </div>
            </div>
        '''))

engine_dropdown.observe(update_engine_info, names='value')

# Initial engine info display
update_engine_info({'new': 'aria2'})

def format_size(size_bytes):
    if size_bytes == 0:
        return "0 B"
    size_names = ["B", "KB", "MB", "GB", "TB"]
    i = 0
    while size_bytes >= 1024 and i < len(size_names) - 1:
        size_bytes /= 1024
        i += 1
    return f"{size_bytes:.2f} {size_names[i]}"

def download_with_aria2(url, output_dir, connections=16, split=16, output_name=None):
    cmd = [
        'aria2c',
        '-x', str(connections),
        '-s', str(split),
        '-j', '5',
        '-k', '10M',
        '--continue=true',
        '--max-tries=10',
        '--retry-wait=5',
        f'--dir={output_dir}',
        '--summary-interval=1',
        '--console-log-level=warn'
    ]
    if output_name:
        cmd.extend(['--out', output_name])
    cmd.append(url)
    return subprocess.run(cmd, capture_output=True, text=True)

def download_with_wget(url, output_dir, output_name=None):
    cmd = ['wget', '--continue', '--progress=bar:force', '--tries=10', '-P', output_dir]
    if output_name:
        cmd.extend(['-O', os.path.join(output_dir, output_name)])
    cmd.append(url)
    return subprocess.run(cmd, capture_output=True, text=True)

def download_with_curl(url, output_dir, output_name=None):
    filename = output_name or os.path.basename(url.split('?')[0]) or 'download'
    output_path = os.path.join(output_dir, filename)
    cmd = ['curl', '-L', '-C', '-', '--retry', '10', '--progress-bar', '-o', output_path, url]
    return subprocess.run(cmd, capture_output=True, text=True)

def download_with_mega(url, output_dir, output_name=None):
    cmd = ['megadl', f'--output={output_dir}/']
    if output_name:
        cmd[1] = f'--output={output_dir}/{output_name}'
    cmd.append(url)
    return subprocess.run(cmd, capture_output=True, text=True)

def start_download(btn):
    urls = [u.value.strip() for u in url_inputs if u.value.strip()]
    if not urls:
        with download_output:
            clear_output()
            display(HTML('<div style="background: #ff6b6b; color: white; padding: 15px; border-radius: 10px;">Please enter at least one URL</div>'))
        return
    
    engine = engine_dropdown.value
    mode = mode_toggle.value
    connections = connections_slider.value
    split = split_slider.value
    output_dir = '/content/download'
    
    with download_output:
        clear_output()
        display(HTML(f'<h4>Starting {len(urls)} download(s) using {ENGINE_INFO[engine]["name"]}...</h4>'))
    
    results = []
    
    def process_single_download(url, index):
        start_time = time.time()
        try:
            if engine == 'aria2':
                result = download_with_aria2(url, output_dir, connections, split)
            elif engine == 'wget':
                result = download_with_wget(url, output_dir)
            elif engine == 'curl':
                result = download_with_curl(url, output_dir)
            elif engine == 'mega':
                result = download_with_mega(url, output_dir)
            
            elapsed = time.time() - start_time
            success = result.returncode == 0
            
            return {
                'url': url,
                'success': success,
                'error': result.stderr if not success else None,
                'elapsed': elapsed,
                'index': index
            }
        except Exception as e:
            return {
                'url': url,
                'success': False,
                'error': str(e),
                'elapsed': time.time() - start_time,
                'index': index
            }
    
    if mode == 'parallel':
        with ThreadPoolExecutor(max_workers=min(len(urls), 3)) as executor:
            futures = {executor.submit(process_single_download, url, i): i for i, url in enumerate(urls)}
            for future in as_completed(futures):
                result = future.result()
                results.append(result)
                update_progress(results, len(urls))
    else:
        for i, url in enumerate(urls):
            result = process_single_download(url, i)
            results.append(result)
            update_progress(results, len(urls))
    
    show_final_results(results)

def update_progress(results, total):
    completed = len(results)
    percentage = (completed / total) * 100
    
    with progress_output:
        clear_output(wait=True)
        display(HTML(f'''
            <div class="progress-container">
                <div style="display: flex; justify-content: space-between; margin-bottom: 10px;">
                    <span>Progress: {completed}/{total}</span>
                    <span>{percentage:.0f}%</span>
                </div>
                <div class="progress-bar-custom">
                    <div class="progress-fill" style="width: {percentage}%;">
                        {percentage:.0f}%
                    </div>
                </div>
            </div>
        '''))

def show_final_results(results):
    success_count = sum(1 for r in results if r['success'])
    fail_count = len(results) - success_count
    
    with download_output:
        clear_output()
        
        html = '''
        <div class="progress-container">
            <h3 style="margin: 0 0 20px 0;">Download Results</h3>
            <div class="stats-grid">
                <div class="stat-card">
                    <div class="stat-value">''' + str(len(results)) + '''</div>
                    <div class="stat-label">Total</div>
                </div>
                <div class="stat-card" style="background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%);">
                    <div class="stat-value">''' + str(success_count) + '''</div>
                    <div class="stat-label">Success</div>
                </div>
                <div class="stat-card" style="background: linear-gradient(135deg, #eb3349 0%, #f45c43 100%);">
                    <div class="stat-value">''' + str(fail_count) + '''</div>
                    <div class="stat-label">Failed</div>
                </div>
            </div>
            <h4 style="margin: 20px 0 10px 0;">Details:</h4>
        '''
        
        for r in results:
            status_class = 'completed' if r['success'] else 'failed'
            status_icon = '✅' if r['success'] else '❌'
            status_text = 'Success' if r['success'] else 'Failed'
            url_short = r['url'][:60] + '...' if len(r['url']) > 60 else r['url']
            
            html += f'''
            <div class="download-item {status_class}">
                <div style="display: flex; justify-content: space-between; align-items: center;">
                    <div>
                        <span>{status_icon} <strong>{status_text}</strong></span>
                        <p style="margin: 5px 0 0 0; font-size: 0.85em; color: #666; font-family: monospace;">{url_short}</p>
                    </div>
                    <span style="font-size: 0.85em; color: #999;">{r['elapsed']:.1f}s</span>
                </div>
            '''
            if r['error']:
                html += f'<p style="margin: 10px 0 0 0; font-size: 0.8em; color: #e74c3c;">Error: {r["error"][:100]}</p>'
            html += '</div>'
        
        html += '</div>'
        display(HTML(html))

# Download button
download_btn = widgets.Button(
    description='Start Download',
    button_style='primary',
    icon='download',
    layout=widgets.Layout(width='200px', height='50px')
)
download_btn.on_click(start_download)

# Build UI
display(HTML('<div class="engine-selector"><h4>Select Download Engine</h4></div>'))
display(widgets.HBox([engine_dropdown], layout=widgets.Layout(justify_content='center')))
display(engine_info_output)

display(HTML('<div class="input-section"><h4>Download URLs (Max 5)</h4></div>'))
for url_input in url_inputs:
    display(url_input)

display(HTML('<div class="input-section"><h4>Configuration</h4></div>'))
display(widgets.HBox([mode_toggle], layout=widgets.Layout(justify_content='center')))

display(HTML('<div class="slider-container"><strong>Aria2 Settings (if selected):</strong></div>'))
display(connections_slider)
display(split_slider)

display(widgets.HBox([download_btn], layout=widgets.Layout(justify_content='center', padding='20px')))
display(progress_output)
display(download_output)

## Cell 4: Transfer to Google Drive

Move or copy downloaded files from `/content/download` to your Google Drive.

In [ ]:
# =============================================================================
# CELL 4: TRANSFER TO GOOGLE DRIVE
# =============================================================================

from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
import subprocess
import os
import shutil
import threading

# CSS Styling
transfer_css = '''
<style>
    .transfer-header {
        background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%);
        color: white;
        padding: 25px;
        border-radius: 15px;
        margin-bottom: 25px;
        text-align: center;
    }
    .transfer-panel {
        background: white;
        border-radius: 12px;
        padding: 25px;
        margin: 15px 0;
        box-shadow: 0 4px 15px rgba(0,0,0,0.1);
    }
    .path-display {
        background: #f8f9fa;
        padding: 15px;
        border-radius: 8px;
        font-family: 'Consolas', monospace;
        margin: 10px 0;
        border-left: 4px solid #11998e;
    }
    .file-list {
        background: #f8f9fa;
        border-radius: 10px;
        padding: 15px;
        max-height: 300px;
        overflow-y: auto;
    }
    .file-item {
        display: flex;
        justify-content: space-between;
        padding: 10px;
        border-bottom: 1px solid #e0e0e0;
        transition: background 0.2s;
    }
    .file-item:hover { background: #e8f5e9; }
    .file-item:last-child { border-bottom: none; }
    .btn-transfer {
        background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%);
        color: white;
        border: none;
        padding: 15px 40px;
        border-radius: 30px;
        font-size: 1.1em;
        font-weight: bold;
        cursor: pointer;
        transition: all 0.3s;
    }
    .btn-transfer:hover {
        transform: translateY(-2px);
        box-shadow: 0 8px 25px rgba(17, 153, 142, 0.4);
    }
    .transfer-progress {
        background: white;
        border-radius: 12px;
        padding: 20px;
        margin: 15px 0;
        box-shadow: 0 4px 15px rgba(0,0,0,0.1);
    }
    .success-message {
        background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%);
        color: white;
        padding: 20px;
        border-radius: 12px;
        text-align: center;
        animation: slideIn 0.5s ease;
    }
    @keyframes slideIn {
        from { opacity: 0; transform: translateY(-20px); }
        to { opacity: 1; transform: translateY(0); }
    }
    .stats-row {
        display: flex;
        justify-content: space-around;
        margin: 20px 0;
    }
    .stat-box {
        text-align: center;
        padding: 15px;
    }
    .stat-number {
        font-size: 2em;
        font-weight: bold;
        color: #11998e;
    }
    .stat-desc {
        color: #666;
        font-size: 0.9em;
    }
</style>
'''

display(HTML(transfer_css))

# Header
display(HTML('''
<div class="transfer-header">
    <h1 style="margin: 0 0 10px 0;">Transfer to Google Drive</h1>
    <p style="margin: 0; opacity: 0.9;">Move or copy downloaded files to persistent storage</p>
</div>
'''))

# Configuration widgets
dest_input = widgets.Text(
    value='/content/drive/MyDrive/Downloads',
    placeholder='Destination path in Drive',
    description='Destination:',
    layout=widgets.Layout(width='100%'),
    style={'description_width': '100px'}
)

mode_radio = widgets.RadioButtons(
    options=[('Move (delete from source)', 'move'), ('Copy (keep in source)', 'copy')],
    value='move',
    description='Mode:',
    layout=widgets.Layout(margin='10px 0')
)

transfer_btn = widgets.Button(
    description='Start Transfer',
    button_style='success',
    icon='exchange',
    layout=widgets.Layout(width='200px', height='50px')
)

# Output containers
files_output = widgets.Output()
transfer_progress_output = widgets.Output()
transfer_result_output = widgets.Output()

SOURCE_DIR = '/content/download'

def format_size(size_bytes):
    if size_bytes == 0:
        return "0 B"
    size_names = ["B", "KB", "MB", "GB", "TB"]
    i = 0
    while size_bytes >= 1024 and i < len(size_names) - 1:
        size_bytes /= 1024
        i += 1
    return f"{size_bytes:.2f} {size_names[i]}"

def scan_source_files():
    files = []
    total_size = 0
    
    if os.path.exists(SOURCE_DIR):
        for item in os.listdir(SOURCE_DIR):
            item_path = os.path.join(SOURCE_DIR, item)
            if os.path.isfile(item_path):
                size = os.path.getsize(item_path)
                files.append({'name': item, 'size': size, 'path': item_path})
                total_size += size
    
    return files, total_size

def refresh_file_list():
    files, total_size = scan_source_files()
    
    with files_output:
        clear_output(wait=True)
        
        if not files:
            display(HTML('''
                <div class="transfer-panel">
                    <h4 style="margin: 0 0 15px 0;">Source Files</h4>
                    <div style="text-align: center; padding: 30px; color: #999;">
                        <p>No files found in /content/download</p>
                        <p style="font-size: 0.85em;">Download files first using Cell 3</p>
                    </div>
                </div>
            '''))
            return
        
        html = f'''
        <div class="transfer-panel">
            <h4 style="margin: 0 0 15px 0;">Source Files ({len(files)} items)</h4>
            <div class="stats-row">
                <div class="stat-box">
                    <div class="stat-number">{len(files)}</div>
                    <div class="stat-desc">Files</div>
                </div>
                <div class="stat-box">
                    <div class="stat-number">{format_size(total_size)}</div>
                    <div class="stat-desc">Total Size</div>
                </div>
            </div>
            <div class="path-display">{SOURCE_DIR}</div>
            <div class="file-list">
        '''
        
        for f in files:
            html += f'''
            <div class="file-item">
                <span>📄 {f['name']}</span>
                <span style="color: #666;">{format_size(f['size'])}</span>
            </div>
            '''
        
        html += '</div></div>'
        display(HTML(html))

def execute_transfer(btn):
    dest = dest_input.value.strip()
    mode = mode_radio.value
    
    if not dest:
        with transfer_result_output:
            clear_output()
            display(HTML('<div style="background: #ff6b6b; color: white; padding: 15px; border-radius: 10px;">Please enter destination path</div>'))
        return
    
    if not os.path.exists('/content/drive'):
        with transfer_result_output:
            clear_output()
            display(HTML('''
                <div style="background: #ff6b6b; color: white; padding: 15px; border-radius: 10px;">
                    Google Drive not mounted! Please run Cell 1 first.
                </div>
            '''))
        return
    
    files, total_size = scan_source_files()
    if not files:
        with transfer_result_output:
            clear_output()
            display(HTML('<div style="background: #ff9800; color: white; padding: 15px; border-radius: 10px;">No files to transfer</div>'))
        return
    
    # Create destination directory
    os.makedirs(dest, exist_ok=True)
    
    with transfer_progress_output:
        clear_output()
        display(HTML('''
            <div class="transfer-progress">
                <div class="loading-container" style="text-align: center; padding: 20px;">
                    <div class="spinner" style="border: 4px solid #f3f3f3; border-top: 4px solid #11998e;
                         border-radius: 50%; width: 40px; height: 40px; animation: spin 1s linear infinite;
                         margin: 0 auto 15px auto;"></div>
                    <p>Transferring files...</p>
                </div>
            </div>
        '''))
    
    success_count = 0
    failed_files = []
    
    for i, f in enumerate(files):
        try:
            src_path = f['path']
            dest_path = os.path.join(dest, f['name'])
            
            if mode == 'move':
                shutil.move(src_path, dest_path)
            else:
                shutil.copy2(src_path, dest_path)
            
            success_count += 1
            
            with transfer_progress_output:
                clear_output(wait=True)
                progress = (i + 1) / len(files) * 100
                display(HTML(f'''
                    <div class="transfer-progress">
                        <p>Transferring: {f['name']}</p>
                        <div class="progress-bar-custom" style="height: 25px;">
                            <div class="progress-fill" style="width: {progress}%;">{int(progress)}%</div>
                        </div>
                        <p style="text-align: center;">{i+1}/{len(files)} files</p>
                    </div>
                '''))
        except Exception as e:
            failed_files.append((f['name'], str(e)))
    
    # Show final results
    with transfer_progress_output:
        clear_output()
    
    with transfer_result_output:
        clear_output()
        
        html = f'''
        <div class="success-message">
            <h3 style="margin: 0 0 15px 0;">Transfer Complete!</h3>
            <div class="stats-row">
                <div class="stat-box" style="color: white;">
                    <div class="stat-number" style="color: white;">{success_count}</div>
                    <div class="stat-desc" style="color: rgba(255,255,255,0.8);">Transferred</div>
                </div>
                <div class="stat-box" style="color: white;">
                    <div class="stat-number" style="color: white;">{len(failed_files)}</div>
                    <div class="stat-desc" style="color: rgba(255,255,255,0.8);">Failed</div>
                </div>
            </div>
            <p style="margin: 15px 0 0 0;">Destination: <code>{dest}</code></p>
        </div>
        '''
        
        if failed_files:
            html += '<div style="margin-top: 15px; background: #ff6b6b; color: white; padding: 15px; border-radius: 10px;">'
            html += '<h4>Failed Files:</h4><ul>'
            for name, error in failed_files:
                html += f'<li>{name}: {error}</li>'
            html += '</ul></div>'
        
        display(HTML(html))
    
    # Refresh file list
    refresh_file_list()

transfer_btn.on_click(execute_transfer)

# Build UI
refresh_file_list()
display(files_output)

display(HTML('<div class="transfer-panel"><h4>Transfer Configuration</h4></div>'))
display(dest_input)
display(mode_radio)

display(widgets.HBox([transfer_btn], layout=widgets.Layout(justify_content='center', padding='20px')))
display(transfer_progress_output)
display(transfer_result_output)

---
## Quick Reference

| Feature | Description |
|---------|-------------|
| **Download Directory** | `/content/download` |
| **Drive Mount Point** | `/content/drive/MyDrive` |
| **Max URLs** | 5 simultaneous downloads |
| **Engines** | Aria2, Wget, cURL, Mega |
| **Max Connections** | 32 (Aria2) |

---
**Cloud Advanced Downloader v2.0** | [GitHub Repository](https://github.com)